In [ ]:
from google.colab import userdata
SERPAPI_API_KEY = userdata.get('SERPAPI_KEY')

In [ ]:
from serpapi import GoogleSearch

In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=1241e855c008a60294399403040bad14e679bf019c62723d3373b0a5c882f7c5
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
def build_search_queries(persona):
    name = persona.get("name", "")
    intro = persona.get("intro", "")
    # add condition to get the specified fields
    # then do data enrichment
    # then perform query enhancing, may use an LLM to further enhance the search query
    # after that get the n top results from serpapi, and checking which matches the most(match different social media urls here only), if still confusion then on the last few results if image is given then match image, or check other information about the 2nd or other specified information
    # validation

    queries = [
        f'"{name}" site:linkedin.com/in',
        f'"{name}" "{intro}" site:linkedin.com/in',
    ]

    if name and len(name.split()) > 0:
        queries.append(f'"{name.split()[0]}" "{intro}" site:linkedin.com/in')

    return queries

In [ ]:

def search_linkedin_profiles(persona, max_results=5):
    queries = build_search_queries(persona)
    seen = set()
    candidates = []

    for query in queries:
        params = {
            "engine": "google",
            "q": query,
            "api_key": SERPAPI_API_KEY,
            "num": max_results
        }

        search = GoogleSearch(params)
        results = search.get_dict()
        for result in results.get("organic_results", []):
            link = result.get("link", "")
            snippet = result.get("snippet", "")
            if "linkedin.com/in/" in link and link not in seen:
                candidates.append({
                    "link": link,
                    "title": result.get("title"),
                    "snippet": snippet
                })
                seen.add(link)

            if len(candidates) >= max_results:
                break

        if len(candidates) >= max_results:
            break

    return candidates

In [ ]:
persona = {
      "name": "Yaren",
      "image": "",
      "intro": "",
      "timezone": "",
      "company_industry": "",
      "company_size": "0-50 Employees",
      "social_profile": []
}

# intro and timezone, query fetching ke waqt pata lag jayega

results = search_linkedin_profiles(persona,20)

for i, r in enumerate(results, 1):
    print(f"[{i}] {r['link']}")
    print(f"Title: {r['title']}")
    print(f"Snippet: {r['snippet']}\n")


[1] https://www.linkedin.com/in/hakanyarenexecutive
Title: Hakan Yaren - Kintetsu World Express
Snippet: Hakan Yaren - Kintetsu World Express - About - Executive with leadership roles in Technology, Strategy, M&A, Product Development, and...

[2] https://www.linkedin.com/in/yaren-%C3%A7%C4%B1nar
Title: Yaren Cinar - Evanston, Illinois, United States
Snippet: Yaren Cinar Student at Northwestern University ÜNLÜ & Co The London School of Economics and Political Science (LSE)

[3] https://www.linkedin.com/in/yaren-mistacoglu-617568250
Title: Yaren Mistacoglu - Strategy Director
Snippet: Yaren Mistacoglu. Data Science & Economics @ UC Berkeley. Undergraduate Marketing Association University of California, Berkeley. Berkeley ...

[4] https://www.linkedin.com/in/yaren-dogan-985bb7230/
Title: Yaren Dogan - Co Director - Hack4Impact UTK
Snippet: Hello everyone, my name is Yaren Dogan (yahr-en doe-an). I was born in Istanbul, Turkey, and I moved to the United States when I was 2 years old.

[5] 

## I should create a class with different methods later on to complete a single big macro task

In [ ]:
#### Step by step implementation
def infer_role_from_size_and_intro(company_size: str, intro: str = "") -> str:
    if intro:
        intro_lower = intro.lower()
        if any(word in intro_lower for word in ["founder", "co-founder", "ceo", "started", "my company", "built", "entrepreneur"]):
            return "Founder"
        elif any(word in intro_lower for word in ["manager", "director", "vp", "team lead"]):
            return "Manager"
        elif "consultant" in intro_lower:
            return "Consultant"
        elif "engineer" in intro_lower:
            return "Engineer"

    # Only fallback to company_size if intro gave no signal
    # later we can apply company enrichment as well
    # should later on add more keywords and semantic search
    if not company_size:
        return ""

    size = company_size.lower()
    if "1" in size or "0" in size:
        return "Founder"
    elif "11" in size:
        return "Co-founder"
    elif "50" in size:
        return "Team Lead"
    elif "200" in size or "500" in size:
        return "Manager"
    elif "1000" in size or "+" in size:
        return "Staff"  # generic, neutral
    return ""


In [ ]:
infer_role_from_size_and_intro("5000","entrepreneur")

'Founder'

## Later on add more different accounts support

## Add more different scrapers for different platforms to get more additional information possible

In [ ]:
import re

def extract_social_usernames(social_profiles):
    usernames_dict = {}

    # Patterns for different social media platforms
    patterns = {
        'twitter': r'(?:twitter\.com|x\.com)/(@?[\w\d\-_]+)',
        'github': r'github\.com/([\w\d\-_]+)',
        'bluesky': r'(?:bsky\.app|bsky\.social)/profile/([\w\d\-_\.]+)',
        'instagram': r'instagram\.com/(?!p/)(@?[\w\d\._]+)',
        'facebook': r'facebook\.com/(?!(?:pages|groups|events)/)([^/\?]+)',
        'linkedin': r'linkedin\.com/in/([\w\d\-_]+)',
        'youtube': r'youtube\.com/(?:c/|channel/|user/|@)([\w\d\-_]+)',
        'tiktok': r'tiktok\.com/(@[\w\d\._]+)',
        'mastodon': r'([\w\d\-_\.]+)@([\w\d\-_\.]+)',  # username@instance.domain
        'threads': r'threads\.net/(@?[\w\d\._]+)'
    }

    for url in social_profiles:
        url = url.strip()

        for platform, pattern in patterns.items():
            match = re.search(pattern, url)
            if match:
                username = match.group(1)

                # Format usernames properly
                if platform == 'twitter' and not username.startswith('@'):
                    username = f'@{username}'
                elif platform == 'tiktok' and not username.startswith('@'):
                    username = f'@{username}'
                elif platform == 'instagram' and not username.startswith('@'):
                    username = f'@{username}'
                elif platform == 'bluesky' and not username.startswith('@'):
                    username = f'@{username}'

                # Add to results dictionary
                if platform not in usernames_dict:
                    usernames_dict[platform] = []
                usernames_dict[platform].append(username)
                break

    return usernames_dict

In [ ]:
profiles = [
        "https://twitter.com/rajavijayach"
    ]

results = extract_social_usernames(profiles)
for platform, users in results.items():
        print(f"{platform}: {users}")

twitter: ['@rajavijayach']


## Generate Name Variants

In [ ]:
!pip install nameparser

In [ ]:
from nameparser import HumanName
from typing import Dict, List, Set, Tuple, Any, Optional

def generate_name_variants(name: str) -> List[str]:
    """Generate multiple name format variants from a full name."""
    if not name:
        return []

    variants = set()
    parsed = HumanName(name)

    # Add first name
    if parsed.first:
        variants.add(parsed.first)

        # Add first + last name combinations
        if parsed.last:
            variants.add(f"{parsed.first} {parsed.last}")
            variants.add(f"{parsed.first[0]}. {parsed.last}")

            # Add middle initial variants if available
            if parsed.middle:
                variants.add(f"{parsed.first} {parsed.middle[0]}. {parsed.last}")
                variants.add(f"{parsed.first} {parsed.middle} {parsed.last}")

    # Add full name and nickname variants
    variants.add(name)
    if parsed.nickname:
        variants.add(parsed.nickname)
        if parsed.last:
            variants.add(f"{parsed.nickname} {parsed.last}")

    return list(variants)

In [ ]:
generate_name_variants("dArshan Surendra Thakare")

['dArshan Thakare',
 'd. Thakare',
 'dArshan',
 'dArshan S. Thakare',
 'dArshan Surendra Thakare']

## Enriching the different user personas

Work on this again indepth

In [ ]:
def enrich_persona_with_api(persona: Dict[str, Any]) -> Dict[str, Any]:
    """
    Optional: Try enriching the persona using PeopleDataLabs or Clearbit
    if name-only data is available.
    Returns the enriched persona or the same if not enriched.
    """
    # Defensive programming
    if not persona or not isinstance(persona, dict):
        return persona

    name = persona.get("name", "")
    if not name or len(name.split()) < 1:
        return persona  # not enough info to enrich

    # Example enrichment
    # result = people_data_labs_enrich(name=name)
    # if result:
    #     persona["company_industry"] = result.get("industry")
    #     persona["intro"] = result.get("job_title")
    #     persona["location"] = result.get("location")

    return persona

## Assigning Scores

In [ ]:
def score_query(query: str, weights: Dict[str, float]) -> float:
    """
    Score a search query based on the weights of its components.
    Applies some penalties and bonuses based on query structure.
    """
    score = sum(weights.values())

    # Bonus for queries with multiple high-value components
    if len(weights) >= 3:
        score *= 1.2

    # Penalty for very long queries
    if len(query.split()) > 10:
        score *= 0.9

    return score

## Final Major function

In [ ]:
def generate_search_queries(persona: Dict[str, Any]) -> List[str]:
    """
    Generate ranked search queries based on a person's information.
    Returns a list of search queries ordered by relevance.
    """
    # Input validation
    if not persona or not isinstance(persona, dict):
        return []

    # Enrich and extract persona data
    persona = enrich_persona_with_api(persona)
    name = persona.get("name", "")
    intro = persona.get("intro", "")
    company = persona.get("company_industry", "")
    company_size = persona.get("company_size", "")
    social_profiles = persona.get("social_profile", [])
    location = persona.get("location", "")

    if not name:  # Name is required
        return []

    queries = []

    # Build base name variants
    name_variants = generate_name_variants(name)

    # Get social usernames
    social_data = extract_social_usernames(social_profiles)
    social_usernames = []
    for platform_usernames in social_data.values():
        social_usernames.extend(platform_usernames)

    # Infer likely role based on company size
    inferred_role = infer_role_from_size_and_intro(company_size,intro)

    # Generate queries with various combinations
    for variant in name_variants:
        # Basic name-only query
        base = f'"{variant}" site:linkedin.com/in'
        queries.append((base, {"name": 1.0}))

        # Name + job title/intro
        if intro:
            queries.append((f'"{variant}" "{intro}" site:linkedin.com/in',
                           {"name": 1.0, "intro": 0.8}))

        # Name + company/industry
        if company:
            queries.append((f'"{variant}" "{company}" site:linkedin.com/in',
                           {"name": 1.0, "company": 0.7}))
            # Name + company + intro for higher specificity
            if intro:
                queries.append((f'"{variant}" "{company}" "{intro}" site:linkedin.com/in',
                               {"name": 1.0, "company": 0.7, "intro": 0.8}))

        # Name + inferred role
        if inferred_role:
            queries.append((f'"{variant}" "{inferred_role}" site:linkedin.com/in',
                           {"name": 1.0, "role": 0.6}))
            # Name + inferred role + company
            if company:
                queries.append((f'"{variant}" "{inferred_role}" "{company}" site:linkedin.com/in',
                               {"name": 1.0, "role": 0.6, "company": 0.7}))

        # Name + location for local professionals
        if location:
            queries.append((f'"{variant}" "{location}" site:linkedin.com/in',
                           {"name": 1.0, "location": 0.5}))

        # Fallback generic role queries
        common_roles = ["Founder", "CEO", "CTO", "Director", "Manager", "Lead"]
        if not intro and not inferred_role:
            for role in common_roles:
                queries.append((f'"{variant}" "{role}" site:linkedin.com/in',
                               {"name": 1.0, "fallback": 0.4}))

    # Social handle-based queries
    for handle in social_usernames:
        queries.append((f'"{handle}" site:linkedin.com/in', {"social": 0.9}))

        # Try to combine social handles with name for better matches
        if name_variants:
            primary_name = name_variants[0]
            queries.append((f'"{handle}" "{primary_name}" site:linkedin.com/in',
                           {"social": 0.9, "name": 1.0}))

    # Deduplicate and rank queries
    seen = set()
    ranked_queries = []

    for query, weights in queries:
        if query not in seen:
            ranked_queries.append((query, score_query(query, weights)))
            seen.add(query)

    # Sort by score in descending order
    ranked_queries.sort(key=lambda x: x[1], reverse=True)

    # Return only the query strings, not their scores
    return [query for query, _ in ranked_queries]

In [ ]:
persona = {
    "name": "Samantha Carter",
    "intro": "Head of Product at Astrogate — building AI systems for enterprise automation",
    "company_industry": "Artificial Intelligence",
    "company_size": "51–200 Employees",
    "timezone": "America/New_York",
    "location": "Brooklyn, NY",
    "social_profile": [
        "https://twitter.com/samcarter_ai",
        "https://github.com/samcarter"
    ],
    "image": "https://images.generated.photos/samantha_carter.jpg"
}

# need to work on company_industry and company_size finding


In [ ]:
generate_search_queries(persona)

['"Samantha Carter" "Founder" "Artificial Intelligence" site:linkedin.com/in',
 '"S. Carter" "Founder" "Artificial Intelligence" site:linkedin.com/in',
 '"Samantha" "Founder" "Artificial Intelligence" site:linkedin.com/in',
 '"Samantha Carter" "Artificial Intelligence" "Head of Product at Astrogate — building AI systems for enterprise automation" site:linkedin.com/in',
 '"S. Carter" "Artificial Intelligence" "Head of Product at Astrogate — building AI systems for enterprise automation" site:linkedin.com/in',
 '"Samantha" "Artificial Intelligence" "Head of Product at Astrogate — building AI systems for enterprise automation" site:linkedin.com/in',
 '"@samcarter_ai" "Samantha Carter" site:linkedin.com/in',
 '"samcarter" "Samantha Carter" site:linkedin.com/in',
 '"Samantha Carter" "Artificial Intelligence" site:linkedin.com/in',
 '"S. Carter" "Artificial Intelligence" site:linkedin.com/in',
 '"Samantha" "Artificial Intelligence" site:linkedin.com/in',
 '"Samantha Carter" "Head of Product 

## So generating advanced search query is done.

## Now first let's just see what all things to add

1. People enrichment if we find the profile at this stage, then it's amazing
2. We will get results: then from those results. rank those on the basis of cosine similarity with the given persona. {{This is also enough}}
3. Then after this some targeted tests like image matching and more, addition

Build this much till today.

4. Making the code better by dividing into multiple small modules
5. Validation engine to validate
6. API creation and deployment
7. Build UI/UX and done

In this section I am writing the code for image matching, so once we get some highly specific queries, if image is provided then we will directly match the images

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8nnzt2eb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8nnzt2eb
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def get_clip_embedding_from_url(image_url):
    try:
        # Add timeout and headers to mimic a browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(image_url, timeout=10, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Check content type to verify it's an image
        content_type = response.headers.get('Content-Type', '')
        if not content_type.startswith('image/'):
            print(f"Warning: URL doesn't return image content type. Got: {content_type}")

        # Debug: save first few bytes to see what we're getting
        content_preview = response.content[:30]
        print(f"Content preview (first 30 bytes): {content_preview}")

        # Try to open the image
        image = Image.open(BytesIO(response.content))
        print(f"Image format: {image.format}, Size: {image.size}, Mode: {image.mode}")

        # Convert to RGB if needed
        if image.mode != "RGB":
            image = image.convert("RGB")

        # Process and get embedding
        processed_image = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = model.encode_image(processed_image)
            embedding /= embedding.norm(dim=-1, keepdim=True)
        return embedding
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except PIL.UnidentifiedImageError as e:
        print(f"Error identifying image: {e}")
        # Try to print the first few bytes to debug
        if 'response' in locals() and hasattr(response, 'content'):
            print(f"First 50 bytes: {response.content[:50]}")
    except Exception as e:
        print(f"Error loading image: {e}")
        import traceback
        traceback.print_exc()
    return None


In [ ]:
def compare_image_similarity_clip(url1, url2):
    print(f"Processing first image from: {url1}")
    emb1 = get_clip_embedding_from_url(url1)

    print(f"Processing second image from: {url2}")
    emb2 = get_clip_embedding_from_url(url2)

    if emb1 is None:
        return {"score": 0.0, "reason": f"First image load or encoding failed"}
    if emb2 is None:
        return {"score": 0.0, "reason": f"Second image load or encoding failed"}

    similarity = (emb1 @ emb2.T).item()  # cosine similarity
    return {
        "score": round(similarity, 3),  # usually 0.0 to 1.0
        "reason": "OK"
    }

In [ ]:
compare_image_similarity_clip("https://upload.wikimedia.org/wikipedia/commons/9/99/Black_square.jpg","https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Black_square.jpg/800px-Black_square.jpg")

Processing first image from: https://upload.wikimedia.org/wikipedia/commons/9/99/Black_square.jpg
Content preview (first 30 bytes): b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x0b\x07\x08\t\x08'
Image format: JPEG, Size: (360, 360), Mode: RGB
Processing second image from: https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Black_square.jpg/800px-Black_square.jpg
Content preview (first 30 bytes): b'\xff\xd8\xff\xdb\x00C\x00\x04\x03\x03\x04\x03\x03\x04\x04\x03\x04\x05\x04\x04\x05\x06\n\x07\x06\x06\x06\x06\r\t'
Image format: JPEG, Size: (800, 800), Mode: RGB


{'score': 0.985, 'reason': 'OK'}

In [ ]:
compare_image_similarity_clip("https://upload.wikimedia.org/wikipedia/commons/3/3f/Fronalpstock_big.jpg","https://upload.wikimedia.org/wikipedia/commons/9/99/Black_square.jpg")

Processing first image from: https://upload.wikimedia.org/wikipedia/commons/3/3f/Fronalpstock_big.jpg
Content preview (first 30 bytes): b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x01,\x01,\x00\x00\xff\xe1\x13DExif\x00\x00'
Image format: JPEG, Size: (10109, 4542), Mode: RGB
Processing second image from: https://upload.wikimedia.org/wikipedia/commons/9/99/Black_square.jpg
Content preview (first 30 bytes): b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x0b\x07\x08\t\x08'
Image format: JPEG, Size: (360, 360), Mode: RGB


{'score': 0.475, 'reason': 'OK'}

## Social Media URL traversal

1. Just fetch the information of different fields required and store in the array
2. Then create a good string with all the information
3. And send this to the gemini api and get back a more informative persona

In [ ]:
import requests

def extract_github_info(username):
    r = requests.get(f"https://api.github.com/users/{username}")
    if r.status_code == 200:
        data = r.json()
        # return {
        #     "name": data.get("name"),
        #     "bio": data.get("bio"),
        #     "location": data.get("location"),
        #     "email": data.get("email"),
        #     "twitter_username": data.get("twitter_username")
        # }
        return data
    return {}

# I want name,company,location,email,bio,twitter_username
# sometimes people mention their linkedin on their different social threads as well


In [ ]:
extract_github_info("Samia35-2973")

{'login': 'Samia35-2973',
 'id': 71200431,
 'node_id': 'MDQ6VXNlcjcxMjAwNDMx',
 'avatar_url': 'https://avatars.githubusercontent.com/u/71200431?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/Samia35-2973',
 'html_url': 'https://github.com/Samia35-2973',
 'followers_url': 'https://api.github.com/users/Samia35-2973/followers',
 'following_url': 'https://api.github.com/users/Samia35-2973/following{/other_user}',
 'gists_url': 'https://api.github.com/users/Samia35-2973/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/Samia35-2973/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/Samia35-2973/subscriptions',
 'organizations_url': 'https://api.github.com/users/Samia35-2973/orgs',
 'repos_url': 'https://api.github.com/users/Samia35-2973/repos',
 'events_url': 'https://api.github.com/users/Samia35-2973/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/Samia35-2973/received_events',
 'type': 'User',
 'user_view_ty

In [ ]:
!pip install nitter-scraper

In [ ]:
from nitter_scraper import NitterScraper

def extract_twitter_info(handle):
    try:
        scraper = NitterScraper()
        user_profile = scraper.get_profile(handle)

        return {
            "username": handle,
            "displayname": user_profile.display_name,
            "bio": user_profile.bio,
            "location": user_profile.location,
            "followers": user_profile.followers,
            "verified": user_profile.is_verified
        }
    except Exception as e:
        return {"error": f"Error extracting Twitter info: {str(e)}"}

ImportError: cannot import name 'soft_unicode' from 'markupsafe' (/usr/local/lib/python3.11/dist-packages/markupsafe/__init__.py)

In [ ]:
extract_twitter_info("imVkohli")

ERROR:snscrape.base:Error retrieving https://twitter.com/imVkohli: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /imVkohli (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/imVkohli failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /imVkohli (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))")), SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /imVkohli (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local i

{'error': 'Error extracting Twitter info: 4 requests to https://twitter.com/imVkohli failed, giving up.'}

In [ ]:
!pip install async_timeout

In [ ]:
!pip install --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Obtaining twint from git+https://github.com/twintproject/twint.git@origin/master#egg=twint
  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to ./src/twint
  Running command git clone --filter=blob:none --quiet https://github.com/twintproject/twint.git /content/src/twint
  Running command git checkout -q origin/master
  Resolved https://github.com/twintproject/twint.git to commit origin/master
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.0/718.0 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.2 MB/s eta 0:00:00
  Created wheel for cchardet: filename=cchardet-2.1

In [ ]:
import twint

def extract_twitter_info_twint(handle):
    try:
        # Configure twint
        c = twint.Config()
        c.Username = handle
        c.User_full = True
        c.Store_object = True
        c.Hide_output = True

        # Run the search
        twint.run.Lookup(c)

        # Get user data
        user = twint.output.users_list[0]

        return {
            "username": user.username,
            "displayname": user.name,
            "bio": user.bio,
            "location": user.location,
            "followers": user.followers,
            "verified": user.verified
        }
    except Exception as e:
        return {"error": f"Error extracting Twitter info: {str(e)}"}

In [ ]:
extract_twitter_info_twint("imVkohli")

{'error': 'Error extracting Twitter info: Could not find the Guest token in HTML'}

In [ ]:
!pip install requests-html

In [ ]:
!pip install lxml_html_clean

In [ ]:
# Install required packages

from requests_html import AsyncHTMLSession

def extract_twitter_info_request(handle):
    try:
        # Create a session
        session = AsyncHTMLSession()

        # Use Twitter's direct URL
        url = f"https://twitter.com/{handle}"

        # Send a GET request with appropriate headers
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept': 'text/html,application/xhtml+xml,application/xml'
        }

        response = session.get(url, headers=headers)

        # Render the JavaScript (this is key for modern Twitter)
        response.html.render(timeout=30)

        # Extract basic info from HTML
        # Note: These selectors may need to be updated as Twitter changes its HTML structure
        displayname_element = response.html.find('div[data-testid="UserName"] span', first=True)
        bio_element = response.html.find('div[data-testid="UserDescription"]', first=True)
        location_element = response.html.find('span[data-testid="UserLocation"]', first=True)
        followers_element = response.html.find('a[href*="/followers"] span', first=True)
        verified_element = response.html.find('svg[aria-label="Verified account"]', first=True)

        displayname = displayname_element.text if displayname_element else "Unknown"
        bio = bio_element.text if bio_element else ""
        location = location_element.text if location_element else ""

        # Parse followers (might need to handle "K" or "M" suffixes)
        followers = 0
        if followers_element:
            followers_text = followers_element.text
            if 'K' in followers_text:
                followers = float(followers_text.replace('K', '')) * 1000
            elif 'M' in followers_text:
                followers = float(followers_text.replace('M', '')) * 1000000
            else:
                followers = int(followers_text.replace(',', ''))

        return {
            "username": handle,
            "displayname": displayname,
            "bio": bio,
            "location": location,
            "followers": followers,
            "verified": verified_element is not None
        }
    except Exception as e:
        return {"error": f"Error extracting Twitter info: {str(e)}"}

In [ ]:
extract_twitter_info_request("imVkohli")

{'error': "Error extracting Twitter info: '_asyncio.Future' object has no attribute 'html'"}

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def extract_twitter_info_soup(handle):
    try:
        # Use a Nitter instance (Twitter alternative frontend)
        url = f"https://nitter.net/{handle}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract information
        display_name = soup.select_one('.profile-card-fullname').text.strip()
        bio = soup.select_one('.profile-bio')
        bio_text = bio.text.strip() if bio else ""
        location = soup.select_one('.profile-location')
        location_text = location.text.strip() if location else ""

        # Extract follower count
        stats = soup.select('.profile-stat-num')
        followers = 0
        if len(stats) >= 3:
            followers_text = stats[2].text.strip()
            followers = int(re.sub(r'[^0-9]', '', followers_text))

        # Check for verified badge
        verified = bool(soup.select_one('.verified-icon'))

        return {
            "username": handle,
            "displayname": display_name,
            "bio": bio_text,
            "location": location_text,
            "followers": followers,
            "verified": verified
        }
    except Exception as e:
        return {"error": f"Error extracting Twitter info: {str(e)}"}

In [ ]:
extract_twitter_info_soup("imVkohli")

{'error': "Error extracting Twitter info: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))"}